## Commonsense

In [6]:
from cot import Collection
coll = Collection.load_thoughtsource_100(load_pregenerated_cots=True)
coll.select_generated_cots(cot_trigger = "kojima-01", api_service='openai',model = 'text-davinci-003')
coll.select_generated_cots(answers="False")
coll.unload_datasets(["med_qa","medmc_qa","strategy_qa",'open_book_qa','worldtree'])
coll
#coll.dump("cs_davinci.json")


| Name           | Train   |   Valid | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [ ]:
"""Went through Annotator"""

In [9]:
annotated = Collection.from_json("cs_davinci_annotated.json")


| Name           | Train   |   Valid | Test   |
|----------------|---------|---------|--------|
| commonsense_qa | -       |     100 | -      |

Not loaded: ['aqua', 'asdiv', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'medmc_qa', 'open_book_qa', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [80]:
"""Moved to annotated files folder"""

ann_1 = Collection.from_json("annotated_ob_davinci.json") #open book
ann_2 = Collection.from_json("annotated_st_davinci.json") #strategu
ann_3 = Collection.from_json("annotated_wt_davinci.json")
ann_4 = Collection.from_json("cs_davinci_annotated.json")

In [72]:
def select_annotated(dataset,split,selection):
    i= 0
    lst = []
    for element in selection[dataset][split]:
        if len(element['generated_cot'])>0:
            if len(element['generated_cot'][0]['annotations'])>0 :
                lst.append(i)
        i+=1
    return lst

In [81]:
ann_1['open_book_qa']['test'] = ann_1['open_book_qa']['test'].select(select_annotated('open_book_qa','test',ann_1))
ann_2['strategy_qa']['train'] = ann_2['strategy_qa']['train'].select(select_annotated('strategy_qa','train',ann_2))
ann_3['worldtree']['test'] = ann_3['worldtree']['test'].select(select_annotated('worldtree','test',ann_3))
ann_4['commonsense_qa']['validation'] = ann_4['commonsense_qa']['validation'].select(select_annotated('commonsense_qa','validation',ann_4))

In [ ]:
annotated_set = ann_4.merge(ann_3).merge(ann_2).merge(ann_1)


In [ ]:
annotated_set.dump('annotated_set.json')